<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-c79ae2d4-448b-8432-666f-b372ae8c8e4d)


In [3]:
tf.config.optimizer.set_jit(True)

In [4]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [8]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                                                          shear_range=10,
                                                          width_shift_range=0.1,
                                                          height_shift_range=0.1, 
                                                          zoom_range=0.2, 
                                                          fill_mode='constant', cval=0)
datagen.fit(x_train)

In [5]:
def gen_model(activation="relu"):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(28, 28, 1)))

  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation, padding="same"))
  model.add(tf.keras.layers.AveragePooling2D(2))
  
  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation))
  model.add(tf.keras.layers.AveragePooling2D())

  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation))
  
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation=activation))
  model.add(tf.keras.layers.Dense(10))
  return model

def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=20,
      restore_best_weights=True,
)

In [9]:
%%time
model = gen_model()
compile_model(model)
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          epochs=1000,
          verbose=1,
          callbacks=[early_stop],
          validation_data=(x_test, y_test),
          validation_batch_size=2048)


Epoch 1/1000
469/469 [==============================] - 17s 36ms/step - loss: 0.3480 - accuracy: 0.8898 - val_loss: 0.0547 - val_accuracy: 0.9822
Epoch 2/1000
469/469 [==============================] - 17s 37ms/step - loss: 0.1103 - accuracy: 0.9664 - val_loss: 0.0331 - val_accuracy: 0.9895
Epoch 3/1000
469/469 [==============================] - 16s 34ms/step - loss: 0.0796 - accuracy: 0.9758 - val_loss: 0.0312 - val_accuracy: 0.9896
Epoch 4/1000
469/469 [==============================] - 17s 37ms/step - loss: 0.0657 - accuracy: 0.9796 - val_loss: 0.0366 - val_accuracy: 0.9873
Epoch 5/1000
469/469 [==============================] - 16s 34ms/step - loss: 0.0547 - accuracy: 0.9830 - val_loss: 0.0253 - val_accuracy: 0.9917
Epoch 6/1000
469/469 [==============================] - 16s 34ms/step - loss: 0.0497 - accuracy: 0.9847 - val_loss: 0.0200 - val_accuracy: 0.9930
Epoch 7/1000
469/469 [==============================] - 16s 34ms/step - loss: 0.0459 - accuracy: 0.9862 - val_loss: 0.0232 -

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.01501705963164568
Test accuracy: 0.9962000250816345
